<a href="https://colab.research.google.com/github/mordor-ai/M1MIASHS_projet_twitter/blob/master/analyse_100M_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!apt-get install libcairo2-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libcairo-script-interpreter2 libpixman-1-dev libxcb-shm0-dev
Suggested packages:
  libcairo2-doc
The following NEW packages will be installed:
  libcairo-script-interpreter2 libcairo2-dev libpixman-1-dev libxcb-shm0-dev
0 upgraded, 4 newly installed, 0 to remove and 25 not upgraded.
Need to get 930 kB of archives.
After this operation, 3,986 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libcairo-script-interpreter2 amd64 1.15.10-2ubuntu0.1 [53.5 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libpixman-1-dev amd64 0.34.0-2 [244 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libxcb-shm0-dev amd64 1.13-2~ubuntu18.04 [6,684 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libcairo2-dev amd64 1.15.10-2ubuntu0.1 [626 kB]
Fetched 

In [5]:
!pip install python-igraph
!pip install pycairo

  Using cached https://files.pythonhosted.org/packages/e8/9d/c8be300fc6b1298559d37a071c3833b0b251e0fff334d2e4c408d5789162/pycairo-1.19.1.tar.gz
  Created wheel for pycairo: filename=pycairo-1.19.1-cp36-cp36m-linux_x86_64.whl size=241267 sha256=20320023c73c060600884ccb7da3ee8770efbb9026b43b834ca6a36caf429740
  Stored in directory: /root/.cache/pip/wheels/34/08/71/ef11d2526c2cb17a581088a89dfe1258f17c911e55ebde0550
Successfully built pycairo


In [6]:
!git clone https://github.com/mordor-ai/M1MIASHS_projet_twitter.git

Cloning into 'M1MIASHS_projet_twitter'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 60 (delta 22), reused 39 (delta 12), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [7]:
cd /content/M1MIASHS_projet_twitter

/content/M1MIASHS_projet_twitter


In [0]:
import numpy as np
import pandas as pd
import igraph
import datetime



def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.0f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])


n_rows = 10000000
compressed = True
tstart = None
tend = None


def start_time():
    global tstart
    tstart = datetime.datetime.now()


def get_delta():
    global tstart
    tend = datetime.datetime.now()
    return tend - tstart


print('====== Profiling results =======')
start_time()
# load the original graph csv
df_edges = pd.read_csv(
    "./files/twitter_100M.csv",
    header='infer',
    sep=' ',
    low_memory=False#,
   # dtype={'source': np.int32, 'target': np.int32}
     #,nrows=n_rows
)
# on renome les colonnes
df_edges.columns = ['source', 'target']
delta1 = get_delta()
print('Time required to load edges: %(delta1)s' % locals())
start_time()

# on charge tout les noeuds
df_nodes = pd.read_csv(
    "./files/twitter-2010-ids.csv",
    header='infer',
    sep=',',
    low_memory=False#,
   # dtype={'node_id': np.int32, 'twitter_id': np.int32}
    #, nrows = n_rows
    #,index_col='node_id'
)
delta2 = get_delta()

print("Dataframe loaded with " + str(len(df_edges)) + "rows.")
print('Time required to load nodes: %(delta2)s' % locals())
print(df_edges.head())
print(df_nodes.head())
#print(df_nodes.to_dict('records'))
#print(df_edges.to_dict('records'))
print("Now loading graph")
start_time()
# chargement du graphe
g = igraph.Graph.DictList(df_nodes.to_dict('records')
                          , df_edges.to_dict('records')
                          , directed=True
                          , vertex_name_attr='node_id'
                          , edge_foreign_keys=('source', 'target')
                          #, iterative=False
                          )
delta2 = get_delta()
print("graph loaded")
print('Time required to load graph: %(delta2)s' % locals())
print("loading graph summary")

start_time()
igraph.summary(g)
delta1 = get_delta()
print('Time required to get summary graph: %(delta1)s' % locals())

# start_time()
# degrees_in = g.degree(mode="in")
# max_deg_in = max(degrees_in)
# print(max_deg_in)
# df_degree_in = [g.vs[idx].attributes() for idx, eb in enumerate(degrees_in) if eb == max_deg_in]
# print(df_degree_in)
# print(" twitter id  having max degree IN: ", df_degree_in)
# delta1 = get_delta()
# print('Time required to get  graph degree in : %(delta1)s' % locals())
#
#
# start_time()
# degrees_out = g.degree(mode="out")
# max_deg_out = max(degrees_out)
# print(max_deg_out)
# df_degree_out = [g.vs[idx].attributes() for idx, eb in enumerate(degrees_out) if eb == max_deg_out]
# print(df_degree_out)
# print(" twitter id  having max degree OUT: ", df_degree_out)
# delta1 = get_delta()
# print('Time required to get  graph degree out : %(delta1)s' % locals())




file_name: str = './files/twitter_' + human_format(n_rows) + '_pickle'+  ('z' if compressed else '')
print("now saving graph to ", file_name)
start_time()
if compressed:
    g.write_picklez(file_name)
else:
    g.write_pickle(file_name)
print("graph saved")
delta4 = get_delta()
print("Time required to save pickle file: %(delta4)s" % locals())
print("====== End of program =======")



====== Profiling results =======
Time required to load edges: 0:00:00.368997
Dataframe loaded with 2086819rows.
Time required to load nodes: 0:00:00.381494
   source  target
0       3     2.0
1       2     3.0
2       5     4.0
3       4     5.0
4       5     6.0
   node_id  twitter_id
0        0  56862681.0
1        1  15767971.0
2        2  22055653.0
3        3  23219170.0
4        4  24503384.0
Now loading graph
graph loaded
Time required to load graph: 0:00:22.360405
loading graph summary
IGRAPH D--- 3149737 2086819 -- 
+ attr: node_id (v), twitter_id (v), source (e), target (e)
Time required to get summary graph: 0:00:00.000178
now saving graph to  ./files/twitter_10M_picklez
graph saved
Time required to save pickle file: 0:01:12.421214
====== End of program =======
